# DELIVERABLE 2: FLIGHT DIFFICULTY SCORE DEVELOPMENT

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

print("Libraries imported successfully.")

Libraries imported successfully.


In [4]:
flight_df = pd.read_csv('Flight Level Data.csv')
bags_df = pd.read_csv('Bag+Level+Data.csv')
pnr_flight_df = pd.read_csv('PNR+Flight+Level+Data.csv')
pnr_remarks_df = pd.read_csv('PNR Remark Level Data.csv')
airports_df = pd.read_csv('Airports Data.csv')
print("All CSV files loaded successfully!")

   

All CSV files loaded successfully!


# Data Merging and Preprocessing Pipeline

In [6]:
# This cell contains the logic to merge and clean all data into a single master DataFrame.

#  1. Clean and Prepare Primary DataFrame 
flight_df.columns = flight_df.columns.str.lower().str.strip()
for col in ['scheduled_departure_datetime_local', 'scheduled_arrival_datetime_local', 'actual_departure_datetime_local', 'actual_arrival_datetime_local', 'scheduled_departure_date_local']:
    flight_df[col] = pd.to_datetime(flight_df[col], errors='coerce')
flight_df.drop_duplicates(subset=['company_id', 'flight_number', 'scheduled_departure_date_local'], inplace=True)
flight_df.dropna(subset=['actual_departure_datetime_local'], inplace=True)

# 2. Prepare and Aggregate Secondary DataFrames 
pnr_flight_df.columns = pnr_flight_df.columns.str.lower().str.strip()
pnr_flight_df['scheduled_departure_date_local'] = pd.to_datetime(pnr_flight_df['scheduled_departure_date_local'], errors='coerce')
bags_df.columns = bags_df.columns.str.lower().str.strip()
bags_df['scheduled_departure_date_local'] = pd.to_datetime(bags_df['scheduled_departure_date_local'], errors='coerce')
pnr_remarks_df.columns = pnr_remarks_df.columns.str.lower().str.strip()

# Aggregate Passenger data
pax_per_pnr = pnr_flight_df.drop_duplicates(subset=['record_locator', 'flight_number', 'scheduled_departure_date_local'])
pax_agg = pax_per_pnr.groupby(['company_id', 'flight_number', 'scheduled_departure_date_local']).agg(
    total_pax=('total_pax', 'sum'),
    child_pax_count=('is_child', lambda x: (x == 'Y').sum()),
    lap_child_count=('lap_child_count', 'sum')
).reset_index()

# Aggregate Bag data
bag_agg = bags_df.groupby(['company_id', 'flight_number', 'scheduled_departure_date_local']).agg(
    total_bags=('bag_tag_unique_number', 'count'),
    transfer_bags=('bag_type', lambda x: x.isin(['Transfer', 'Hot']).sum()),
    hot_transfer_bags=('bag_type', lambda x: (x == 'Hot').sum())
).reset_index()

# Aggregate SSR data
pnr_with_date = pnr_flight_df[['record_locator', 'company_id', 'flight_number', 'scheduled_departure_date_local']].drop_duplicates()
remarks_to_merge = pnr_remarks_df.drop(columns=['flight_number', 'pnr_creation_date'])
remarks_with_date = pd.merge(remarks_to_merge, pnr_with_date, on='record_locator', how='left')
ssr_agg = remarks_with_date.groupby(['company_id', 'flight_number', 'scheduled_departure_date_local']).size().reset_index(name='ssr_count')

#  3. Merge into Master DataFrame 
master_df = pd.merge(flight_df, pax_agg, on=['company_id', 'flight_number', 'scheduled_departure_date_local'], how='left')
master_df = pd.merge(master_df, bag_agg, on=['company_id', 'flight_number', 'scheduled_departure_date_local'], how='left')
master_df = pd.merge(master_df, ssr_agg, on=['company_id', 'flight_number', 'scheduled_departure_date_local'], how='left')

#  4. Final Cleaning 
cols_to_fill = ['total_pax', 'child_pax_count', 'lap_child_count', 'total_bags', 'transfer_bags', 'hot_transfer_bags', 'ssr_count']
for col in cols_to_fill:
    master_df[col] = master_df[col].fillna(0)

print("Master DataFrame created successfully.")
print(f"Shape of master_df: {master_df.shape}")

Master DataFrame created successfully.
Shape of master_df: (8063, 22)


# Feature Engineering

In [7]:
print("Engineering features.")

#  Feature Category: Ground Time Constraints 
master_df['f_ground_time_pressure'] = master_df['minimum_turn_minutes'] / master_df['scheduled_ground_time_minutes']

#  Feature Category: Flight-Specific Characteristics
# 1. Passenger Load
master_df['f_load_factor'] = master_df['total_pax'] / master_df['total_seats']
# 2. Children on board
master_df['f_child_ratio'] = (master_df['child_pax_count'] + master_df['lap_child_count']) / master_df['total_pax']
# 3. Baggage Load
master_df['f_bags_per_pax'] = master_df['total_bags'] / master_df['total_pax']
# 4. Baggage Complexity (Transfer bags are harder to handle)
master_df['f_transfer_bag_ratio'] = master_df['transfer_bags'] / master_df['total_bags']
# 5. Aircraft Size (Wide-body planes require more resources)
wide_bodies = ['B767', 'B777', 'B787', 'A330', 'A340', 'A350', 'A380']
master_df['f_is_wide_body'] = master_df['fleet_type'].apply(lambda x: 1 if any(wb in str(x) for wb in wide_bodies) else 0)
# 6. Haul (Longer flights can have more complex catering, cargo, and crew needs)
master_df['flight_duration_hours'] = (master_df['scheduled_arrival_datetime_local'] - master_df['scheduled_departure_datetime_local']).dt.total_seconds() / 3600
haul_bins = [-1, 3, 6, 24] # up to 3h=Short, 3-6h=Medium, >6h=Long
haul_labels = [0, 1, 2] # Numerical representation for the model
master_df['f_haul_category'] = pd.cut(master_df['flight_duration_hours'], bins=haul_bins, labels=haul_labels, right=True)

#  Feature Category: Passenger Service Needs 
# 1. SSRs 
master_df.rename(columns={'ssr_count': 'f_ssr_count'}, inplace=True)
# 2. Hot bags indicate tight connections, a proxy for passenger connection stress
master_df.rename(columns={'hot_transfer_bags': 'f_hot_transfer_bags'}, inplace=True)

# Additional Features
# 1. Departure Time (Morning/Afternoon/Evening rush hours are more complex)
master_df['f_departure_hour'] = master_df['scheduled_departure_datetime_local'].dt.hour
# 2. International flights have extra documentation/security requirements
airports_df.columns = airports_df.columns.str.lower().str.strip()
master_df = pd.merge(master_df, airports_df, left_on='scheduled_arrival_station_code', right_on='airport_iata_code', how='left')
master_df['f_is_international'] = (master_df['iso_country_code'] != 'US').astype(int)

# Final Cleanup 
# Clean up any NaNs or Infs created during division
feature_cols_to_clean = [
    'f_ground_time_pressure', 'f_load_factor', 'f_child_ratio', 'f_bags_per_pax',
    'f_transfer_bag_ratio'
]
master_df[feature_cols_to_clean] = master_df[feature_cols_to_clean].replace([np.inf, -np.inf], np.nan).fillna(0)
master_df['f_haul_category'] = master_df['f_haul_category'].cat.codes # Convert categorical to numeric

print("Feature engineering complete.")

Engineering features.
Feature engineering complete.


# Scoring, Ranking, and Classification

In [13]:
# 1. Define Feature Set and Weights 
feature_cols = [
    'f_ground_time_pressure', 'f_transfer_bag_ratio', 'f_ssr_count', 'f_load_factor',
    'f_is_wide_body', 'f_bags_per_pax', 'f_haul_category', 'f_is_international',
    'f_hot_transfer_bags', 'f_child_ratio'
]
weights = {
    'f_ground_time_pressure': 0.25, 'f_transfer_bag_ratio': 0.20, 'f_ssr_count': 0.15,
    'f_load_factor': 0.10, 'f_is_wide_body': 0.07, 'f_bags_per_pax': 0.05,
    'f_haul_category': 0.05, 'f_is_international': 0.05, 'f_hot_transfer_bags': 0.05,
    'f_child_ratio': 0.03
}

# 2. Calculate Scores Day by Day 
all_days_scored = []
for date, daily_df in master_df.groupby(master_df['scheduled_departure_date_local'].dt.date):
    daily_df_processed = daily_df.copy()
    for col in feature_cols:
        if daily_df_processed[col].max() == daily_df_processed[col].min():
            daily_df_processed[f'{col}_norm'] = 0.0
        else:
            scaler = MinMaxScaler()
            daily_df_processed[f'{col}_norm'] = scaler.fit_transform(daily_df_processed[[col]])

    daily_df_processed['difficulty_score'] = 0
    for feature, weight in weights.items():
        daily_df_processed['difficulty_score'] += daily_df_processed[f'{feature}_norm'] * weight
    all_days_scored.append(daily_df_processed)

scored_df = pd.concat(all_days_scored)

# 3. Final Ranking & Classification 
scored_df['difficulty_score'] = (MinMaxScaler(feature_range=(1, 100)).fit_transform(scored_df[['difficulty_score']]))
scored_df['daily_rank'] = scored_df.groupby(scored_df['scheduled_departure_date_local'].dt.date)['difficulty_score'].rank(method='first', ascending=False)

def classify_flight(rank, total_flights):
    if rank <= total_flights * 0.2: return 'Difficult'
    elif rank <= total_flights * 0.7: return 'Medium'
    else: return 'Easy'

group_sizes = scored_df.groupby(scored_df['scheduled_departure_date_local'].dt.date)['daily_rank'].transform('max')
scored_df['difficulty_class'] = scored_df.apply(lambda row: classify_flight(row['daily_rank'], group_sizes[row.name]), axis=1)

print("Flight Difficulty Score calculated, ranked, and classified.")

Flight Difficulty Score calculated, ranked, and classified.


# Review and Save Final Output for Submission

In [18]:
group_name = "Hacks_on_Crack"  

# --- 2. Select Columns for Final Output as per Submission Guidelines ---
flight_details_cols = [
    'company_id', 'flight_number', 'scheduled_departure_date_local',
    'scheduled_departure_station_code', 'scheduled_arrival_station_code', 'fleet_type'
]
score_cols = ['difficulty_score', 'daily_rank', 'difficulty_class']
final_output_df = scored_df[flight_details_cols + feature_cols + score_cols]

# --- 3. Save to CSV in the Required Format ---
output_filename = f"test_{group_name}.csv"
final_output_df.to_csv(output_filename, index=False)

print("\n----------------------------------------------------")
print(f"SUCCESS: Your submission file has been generated!")
print(f"Filename: {output_filename}")
print("----------------------------------------------------")
print("\nFile Preview (First 5 Rows):")
print(final_output_df.head())


----------------------------------------------------
SUCCESS: Your submission file has been generated!
Filename: test_Hacks_on_Crack.csv
----------------------------------------------------

File Preview (First 5 Rows):
   company_id  flight_number scheduled_departure_date_local  \
35         OO           5564                     2025-08-01   
42         UA           1899                     2025-08-01   
48         UA           2189                     2025-08-01   
69         G7           4590                     2025-08-01   
76         UA            224                     2025-08-01   

   scheduled_departure_station_code scheduled_arrival_station_code fleet_type  \
35                              ORD                            SBN    CRJ-550   
42                              ORD                            YYC   A319-100   
48                              ORD                            LGA  B737-MAX8   
69                              ORD                            MDT    CRJ-55